<a href="https://colab.research.google.com/github/HariNurd/Hadoop-homework/blob/main/Homework_Hadoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!java -version

!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!update-alternatives --set javac /usr/lib/jvm/java-8-openjdk-amd64/bin/javac
!update-alternatives --set jps /usr/lib/jvm/java-8-openjdk-amd64/bin/jps
!java -version

#Finding the default Java path
!readlink -f /usr/bin/java | sed "s:bin/java::"
!apt-get install openssh-server -qq > /dev/null
!service ssh start

!grep Port /etc/ssh/sshd_config

#Creating a new rsa key pair with empty password
!ssh-keygen -t rsa -P "" -f ~/.ssh/id_rsa <<< y

# See id_rsa.pub content
!more /root/.ssh/id_rsa.pub

#Copying the key to autorized keys
!cat $HOME/.ssh/id_rsa.pub > $HOME/.ssh/authorized_keys
#Changing the permissions on the key
!chmod 0600 ~/.ssh/authorized_keys

#Conneting with the local machine
!ssh -o StrictHostKeyChecking=no localhost uptime

# https://archive.apache.org/dist/hadoop/common/hadoop-3.2.3/hadoop-3.2.3.tar.gz
# mirror download: http://mirrors.cloud.aliyuncs.com/apache/hadoop/common/hadoop-3.2.4/hadoop-3.2.4.tar.gz

#Downloading Hadoop 3.2.4
!wget -q https://downloads.apache.org/hadoop/common/hadoop-3.2.4/hadoop-3.2.4.tar.gz

#Untarring the file
!sudo tar -xzf hadoop-3.2.4.tar.gz
#Removing the tar file
!rm hadoop-3.2.4.tar.gz


#Copying the hadoop files to user/local
!cp -r hadoop-3.2.4/ /usr/local/
#-r copy directories recursively

#Adding JAVA_HOME directory to hadoop-env.sh file
!sed -i '/export JAVA_HOME=/a export JAVA_HOME=\/usr\/lib\/jvm\/java-8-openjdk-amd64' /usr/local/hadoop-3.2.4/etc/hadoop/hadoop-env.sh

import os
#Creating environment variables
#Creating Hadoop home variable

os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.2.4"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre"
os.environ["PATH"] += f'{os.environ["JAVA_HOME"]}/bin:{os.environ["JRE_HOME"]}/bin:{os.environ["HADOOP_HOME"]}/sbin'

#Dowloading text example to use as input
!wget -q https://www.mirrorservice.org/sites/ftp.ibiblio.org/pub/docs/books/gutenberg/1/0/101/101.txt

openjdk version "11.0.26" 2025-01-21
OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/bin/javac to provide /usr/bin/javac (javac) in manual mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/bin/jps to provide /usr/bin/jps (jps) in manual mode
openjdk version "1.8.0_442"
OpenJDK Runtime Environment (build 1.8.0_442-8u442-b06~us1-0ubuntu1~22.04-b06)
OpenJDK 64-Bit Server VM (build 25.442-b06, mixed mode)
/usr/lib/jvm/java-8-openjdk-amd64/jre/
 * Starting OpenBSD Secure Shell server sshd
   ...done.
#Port 22
#GatewayPorts no
Generating public/private rsa key pair.
Created directory '/root/.ssh'.
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in

In [2]:
#Adding required property to core-site.xml file
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/core-site.xml
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
          <name>fs.defaultFS</name>
          <value>hdfs://localhost:9000</value>
          <description>Where HDFS NameNode can be found on the network</description>
  </property>
</configuration>
EOF

In [3]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/hdfs-site.xml
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
<property>
    <name>dfs.replication</name>
    <value>1</value>
  </property>

</configuration>
EOF

In [4]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/mapred-site.xml
<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
<property>
    <name>mapreduce.framework.name</name>
    <value>yarn</value>
  </property>
  <property>
    <name>mapreduce.application.classpath</name>
    <value>$HADOOP_HOME/share/hadoop/mapreduce/*:$HADOOP_HOME/share/hadoop/mapreduce/lib/*</value>
  </property>

</configuration>
EOF

In [5]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/yarn-site.xml
<?xml version="1.0"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->
<configuration>
<property>
    <description>The hostname of the RM.</description>
    <name>yarn.resourcemanager.hostname</name>
    <value>localhost</value>
  </property>
  <property>
    <name>yarn.nodemanager.aux-services</name>
    <value>mapreduce_shuffle</value>
  </property>
  <property>
    <name>yarn.nodemanager.env-whitelist</name>
    <value>JAVA_HOME,HADOOP_COMMON_HOME,HADOOP_HDFS_HOME,HADOOP_CONF_DIR,CLASSPATH_PREPEND_DISTCACHE,HADOOP_YARN_HOME,HADOOP_HOME,PATH,LANG,TZ,HADOOP_MAPRED_HOME</value>
  </property>

<!-- Site specific YARN configuration properties -->

</configuration>
EOF

In [6]:
!cat $HADOOP_HOME/etc/hadoop/core-site.xml

<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
          <name>fs.defaultFS</name>
          <value>hdfs://localhost:9000</value>
          <description>Where HDFS NameNode can be found on the network</description>
  </property>
</configuration>


In [7]:
!$HADOOP_HOME/bin/hdfs namenode -format

#Creating other necessary enviroment variables before starting nodes
os.environ["HDFS_NAMENODE_USER"] = "root"
os.environ["HDFS_DATANODE_USER"] = "root"
os.environ["HDFS_SECONDARYNAMENODE_USER"] = "root"
os.environ["YARN_RESOURCEMANAGER_USER"] = "root"
os.environ["YARN_NODEMANAGER_USER"] = "root"

#Launching hdfs deamons
!$HADOOP_HOME/sbin/start-dfs.sh

#Launching yarn deamons
#nohup causes a process to ignore a SIGHUP signal
!nohup $HADOOP_HOME/sbin/start-yarn.sh

#Listing the running deamons
!jps

2025-03-13 14:22:41,698 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = 134fa3287f1f/172.28.0.12
STARTUP_MSG:   args = [-format]
STARTUP_MSG:   version = 3.2.4
STARTUP_MSG:   classpath = /usr/local/hadoop-3.2.4/etc/hadoop:/usr/local/hadoop-3.2.4/share/hadoop/common/lib/j2objc-annotations-1.1.jar:/usr/local/hadoop-3.2.4/share/hadoop/common/lib/asm-5.0.4.jar:/usr/local/hadoop-3.2.4/share/hadoop/common/lib/kerb-core-1.0.1.jar:/usr/local/hadoop-3.2.4/share/hadoop/common/lib/jsch-0.1.55.jar:/usr/local/hadoop-3.2.4/share/hadoop/common/lib/commons-math3-3.1.1.jar:/usr/local/hadoop-3.2.4/share/hadoop/common/lib/kerb-admin-1.0.1.jar:/usr/local/hadoop-3.2.4/share/hadoop/common/lib/jetty-util-ajax-9.4.43.v20210629.jar:/usr/local/hadoop-3.2.4/share/hadoop/common/lib/kerb-simplekdc-1.0.1.jar:/usr/local/hadoop-3.2.4/share/hadoop/common/lib/dnsjava-2.1.7.jar:/usr/local/hadoop-3.2.4/share/hadoop/co

In [8]:
#Report the basic file system information and statistics
!$HADOOP_HOME/bin/hdfs dfsadmin -report

Configured Capacity: 115658190848 (107.72 GB)
Present Capacity: 73759883264 (68.69 GB)
DFS Remaining: 73759858688 (68.69 GB)
DFS Used: 24576 (24 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 127.0.0.1:9866 (localhost)
Hostname: 134fa3287f1f
Decommission Status : Normal
Configured Capacity: 115658190848 (107.72 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 41881530368 (39.01 GB)
DFS Remaining: 73759858688 (68.69 GB)
DFS Used%: 0.00%
DFS Remaining%: 63.77%
Configured Cache Capacity: 0 (0 B)
Cache

In [9]:
#Dowloading text example to use as input (if it has not been donwloaded yet)
!wget -q https://raw.githubusercontent.com/ayyoubmaul/hadoop-docker/refs/heads/main/data/uud45.txt --output-document=/content/101.txt

#Creating directory in HDFS
!$HADOOP_HOME/bin/hdfs dfs -mkdir /word_count
#Coping file from local file system to HDFS
!$HADOOP_HOME/bin/hdfs dfs -put /content/101.txt /word_count

#Exploring Hadoop folder
!$HADOOP_HOME/bin/hdfs dfs -ls /word_count

# Run MapReduce Example using JAVA
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.2.4.jar wordcount /word_count/101.txt /word_count/output_wordcount/

Found 1 items
-rw-r--r--   1 root supergroup       1412 2025-03-13 14:23 /word_count/101.txt
2025-03-13 14:23:25,826 INFO client.RMProxy: Connecting to ResourceManager at localhost/127.0.0.1:8032
2025-03-13 14:23:26,530 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1741875788214_0001
2025-03-13 14:23:26,873 INFO input.FileInputFormat: Total input files to process : 1
2025-03-13 14:23:27,442 INFO mapreduce.JobSubmitter: number of splits:1
2025-03-13 14:23:27,864 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1741875788214_0001
2025-03-13 14:23:27,868 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-03-13 14:23:28,254 INFO conf.Configuration: resource-types.xml not found
2025-03-13 14:23:28,255 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
2025-03-13 14:23:28,806 INFO impl.YarnClientImpl: Submitted application application_1741875788214_0001
2025-03-13 14:23:28,854 INFO mapreduc

In [10]:
!$HADOOP_HOME/bin/hdfs dfs --help

--help: Unknown command
Usage: hadoop fs [generic options]
	[-appendToFile <localsrc> ... <dst>]
	[-cat [-ignoreCrc] <src> ...]
	[-checksum <src> ...]
	[-chgrp [-R] GROUP PATH...]
	[-chmod [-R] <MODE[,MODE]... | OCTALMODE> PATH...]
	[-chown [-R] [OWNER][:[GROUP]] PATH...]
	[-copyFromLocal [-f] [-p] [-l] [-d] [-t <thread count>] [-q <thread pool queue size>] <localsrc> ... <dst>]
	[-copyToLocal [-f] [-p] [-crc] [-ignoreCrc] [-t <thread count>] [-q <thread pool queue size>] <src> ... <localdst>]
	[-count [-q] [-h] [-v] [-t [<storage type>]] [-u] [-x] [-e] <path> ...]
	[-cp [-f] [-p | -p[topax]] [-d] [-t <thread count>] [-q <thread pool queue size>] <src> ... <dst>]
	[-createSnapshot <snapshotDir> [<snapshotName>]]
	[-deleteSnapshot <snapshotDir> <snapshotName>]
	[-df [-h] [<path> ...]]
	[-du [-s] [-h] [-v] [-x] <path> ...]
	[-expunge [-immediate]]
	[-find <path> ... <expression> ...]
	[-get [-f] [-p] [-crc] [-ignoreCrc] [-t <thread count>] [-q <thread pool queue size>] <src> ... <localds

In [11]:
#Exploring the created output directory
#part-r-00000 contains the actual ouput
!$HADOOP_HOME/bin/hdfs dfs -ls /word_count/output_wordcount

Found 2 items
-rw-r--r--   1 root supergroup          0 2025-03-13 14:23 /word_count/output_wordcount/_SUCCESS
-rw-r--r--   1 root supergroup       1276 2025-03-13 14:23 /word_count/output_wordcount/part-r-00000


In [12]:
#Printing out first 50 lines
!$HADOOP_HOME/bin/hdfs dfs -cat /word_count/output_wordcount/part-r-00000

# copy file from HDFS to local disk
!$HADOOP_HOME/bin/hdfs dfs -copyToLocal /word_count/output_wordcount/part-r-00000 /content/output_mapred.txt

:	1
Allah	1
Atas	1
Bahwa	1
Dan	1
Dasar	1
Esa,	1
Indonesia	8
Indonesia,	3
Indonesia.	1
Kebangsaan	1
Kemerdekaan	1
Kemudian	1
Ketuhanan	1
Kuasa	1
Maha	2
Negara	3
Pemerintah	1
Republik	1
Undang-Undang	1
Yang	2
abadi	1
adil	2
bagi	1
bangsa	2
bangsa,	1
bebas,	1
beradab,	1
berbahagia	1
berdasar	1
berdasarkan	1
berdaulat,	1
berkat	1
berkedaulatan	1
berkehidupan	1
bersatu,	1
dalam	3
dan	10
darah	1
daripada	1
dengan	6
depan	1
diatas	1
didorongkan	1
dihapuskan,	1
dipimpin	1
disusunlah	1
dunia	2
gerbang	1
hak	1
harus	1
hikmat	1
ialah	1
ikut	1
ini	1
itu	3
itu,	1
karena	1
ke	1
keadilan	2
kebangsaan	1
kebijaksanaan	1
kehidupan	1
keinginan	1
kemanusiaan	1
kemerdekaan	3
kemerdekaan,	1
kemerdekaannya.	1
kepada	2
kerakyatan	1
kesejahteraan	1
ketertiban	1
luhur,	1
maka	3
makmur.	1
melaksanakan	1
melindungi	1
memajukan	1
membentuk	1
mencerdaskan	1
mengantarkan	1
menyatakan	1
merdeka,	1
mewujudkan	1
negara	1
oleh	3
penjajahan	1
perdamaian	1
pergerakan	1
perikeadilan.	1
perikemanusiaan	1
perjuangan	1
permus

In [13]:
#Exploring Hadoop utilities available
!ls $HADOOP_HOME/share/hadoop/tools/lib/

aliyun-java-sdk-core-4.5.10.jar      hadoop-gridmix-3.2.4.jar
aliyun-java-sdk-kms-2.11.0.jar	     hadoop-kafka-3.2.4.jar
aliyun-java-sdk-ram-3.1.0.jar	     hadoop-openstack-3.2.4.jar
aliyun-sdk-oss-3.13.0.jar	     hadoop-resourceestimator-3.2.4.jar
aws-java-sdk-bundle-1.11.901.jar     hadoop-rumen-3.2.4.jar
azure-data-lake-store-sdk-2.2.9.jar  hadoop-sls-3.2.4.jar
azure-keyvault-core-1.0.0.jar	     hadoop-streaming-3.2.4.jar
azure-storage-7.0.0.jar		     ini4j-0.5.4.jar
hadoop-aliyun-3.2.4.jar		     jdom2-2.0.6.jar
hadoop-archive-logs-3.2.4.jar	     kafka-clients-2.8.1.jar
hadoop-archives-3.2.4.jar	     lz4-java-1.7.1.jar
hadoop-aws-3.2.4.jar		     ojalgo-43.0.jar
hadoop-azure-3.2.4.jar		     opentracing-api-0.33.0.jar
hadoop-azure-datalake-3.2.4.jar      opentracing-noop-0.33.0.jar
hadoop-datajoin-3.2.4.jar	     opentracing-util-0.33.0.jar
hadoop-distcp-3.2.4.jar		     org.jacoco.agent-0.8.5-runtime.jar
hadoop-extras-3.2.4.jar		     wildfly-openssl-1.0.7.Final.jar
hadoop-fs2img-3.2.4.

In [14]:
#Dowloading text example to use as input (if it has not been donwloaded yet)
!wget -q https://raw.githubusercontent.com/ayyoubmaul/hadoop-docker/refs/heads/main/data/uud45.txt

In [15]:
#Creating directory in HDFS
!$HADOOP_HOME/bin/hdfs dfs -mkdir /word_count_with_python

In [16]:
#Copying the file from local file system to Hadoop distributed file system (HDFS)
!$HADOOP_HOME/bin/hdfs dfs -put /content/uud45.txt /word_count_with_python

In [17]:
!$HADOOP_HOME/bin/hdfs dfs -ls /word_count_with_python

Found 1 items
-rw-r--r--   1 root supergroup       1412 2025-03-13 14:24 /word_count_with_python/uud45.txt


In [18]:
%%writefile mapper.py

#!/usr/bin/env python

#'#!' is known as shebang and used for interpreting the script

# import sys because we need to read and write data to STDIN and STDOUT
import sys

# reading entire line from STDIN (standard input)
for line in sys.stdin:
  # to remove leading and trailing whitespace
  ###
  ### "sadsadas dsdasda pergi pergi"
  ### sadsadas 1
  ### sadsadas 1
  ### pergi 1
  ## pergi 1
  line = line.strip()
  # split the line into words, output data type list
  words = line.split()

  # we are looping over the words array and printing the word
  # with the count of 1 to the STDOUT
  for word in words:
    # write the results to STDOUT (standard output);
    # what we output here will be the input for the
    # Reduce step, i.e. the input for reducer.py
    print('%s\t%s' % (word, 1))

Writing mapper.py


In [19]:
%%writefile reducer.py

#!/usr/bin/env python

from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

# read the entire line from STDIN
for line in sys.stdin:
  # remove leading and trailing whitespace
  line = line.strip()
  # splitting the data on the basis of tab we have provided in mapper.py
  word, count = line.split('\t', 1)
  # convert count (currently a string) to int
  # aku sedang belajar hadoop
  ## output dari mapper
  # aku, 1
  # sedang, 1
  # sedang, 1
  # belajar, 1
  # hadoop, 1
  try:
    count = int(count)
  except ValueError:
    # count was not a number, so silently
    # ignore/discard this line
    continue

  # this IF-switch only works because Hadoop sorts map output
  # by key (here: word) before it is passed to the reducer
  if current_word == word:
    current_count += count
  else:
    if current_word: #to not print current_word=None
      # write result to STDOUT
      print('%s\t%s' % (current_word, current_count))
    current_count = count
    current_word = word

# do not forget to output the last word if needed!
if current_word == word:
  print('%s\t%s' % (current_word, current_count))

Writing reducer.py


In [20]:
#Changing the permissions of the files
!chmod 777 /content/mapper.py /content/reducer.py
#Setting 777 permissions to a file or directory means that it will be readable, writable and executable by all users

In [21]:

#Running MapReduce programs in hadoop
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.2.4.jar \
  -input /word_count_with_python/uud45.txt \
  -output /word_count_with_python/output \
  -mapper "python /content/mapper.py" \
  -reducer "python /content/reducer.py"

packageJobJar: [/tmp/hadoop-unjar3496160480490043802/] [] /tmp/streamjob7032202548048098120.jar tmpDir=null
2025-03-13 14:24:12,859 INFO client.RMProxy: Connecting to ResourceManager at localhost/127.0.0.1:8032
2025-03-13 14:24:13,186 INFO client.RMProxy: Connecting to ResourceManager at localhost/127.0.0.1:8032
2025-03-13 14:24:13,509 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1741875788214_0002
2025-03-13 14:24:13,875 INFO mapred.FileInputFormat: Total input files to process : 1
2025-03-13 14:24:14,779 INFO mapreduce.JobSubmitter: number of splits:2
2025-03-13 14:24:15,009 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1741875788214_0002
2025-03-13 14:24:15,011 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-03-13 14:24:15,276 INFO conf.Configuration: resource-types.xml not found
2025-03-13 14:24:15,276 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
2025-03-13 14:24:15,3

In [22]:
#Exploring the created output directory
#part-r-00000 contains the actual ouput
!$HADOOP_HOME/bin/hdfs dfs -ls /word_count_with_python/output

Found 2 items
-rw-r--r--   1 root supergroup          0 2025-03-13 14:24 /word_count_with_python/output/_SUCCESS
-rw-r--r--   1 root supergroup       1276 2025-03-13 14:24 /word_count_with_python/output/part-00000


In [23]:
#Printing out first 50 lines
!$HADOOP_HOME/bin/hdfs dfs -cat /word_count_with_python/output/part-00000 | head -50

:	1
Allah	1
Atas	1
Bahwa	1
Dan	1
Dasar	1
Esa,	1
Indonesia	8
Indonesia,	3
Indonesia.	1
Kebangsaan	1
Kemerdekaan	1
Kemudian	1
Ketuhanan	1
Kuasa	1
Maha	2
Negara	3
Pemerintah	1
Republik	1
Undang-Undang	1
Yang	2
abadi	1
adil	2
bagi	1
bangsa	2
bangsa,	1
bebas,	1
beradab,	1
berbahagia	1
berdasar	1
berdasarkan	1
berdaulat,	1
berkat	1
berkedaulatan	1
berkehidupan	1
bersatu,	1
dalam	3
dan	10
darah	1
daripada	1
dengan	6
depan	1
diatas	1
didorongkan	1
dihapuskan,	1
dipimpin	1
disusunlah	1
dunia	2
gerbang	1
hak	1


In [24]:
# copy file hdfs-wordcount.txt dari HDFS ke local
!$HADOOP_HOME/bin/hdfs dfs -copyToLocal /word_count_with_python/output/part-00000 /content/hdfs-wordcount.txt